# Lume: Workshop Editing & Deployment

This cookbook walks you through the WorkShop workflow and how you can edit your pipeline easily. 

### Overview


- **Executing and Deploying Workshop Edits:** Learn how to access and interpret the result specification and associated mappings that are returned after a job execution. This section will guide you through understanding the structure of the results, including spec, mappings, and the confidence score.


In [1]:
pip install lume-py # install Lume's Python SDK

### Set up your lume-api-key

First let's define our api-key for making calls to the Lume API.

In [ ]:
import lume_py as lume

lume.set_api_key("...")  # sets the api-key

In [ ]:
import os
import json

target_data_path = os.path.join(os.getcwd(), "./data/target.json")
with open(target_data_path) as f:
    target_data = json.load(f)

source_data_path = os.path.join(os.getcwd(), "./data/source.json")
with open(source_data_path) as f:
    source_data = json.load(f)

### Understanding the Workshop

A workshop must always be connected to a Job, and thus you create Workshops based on the job. This is because when making edits, the session must have a designated Mapper, source data, and target schema. Thus, a workshop session grounds to a job and its corresponding pipeline.

You can create new workshops for a job to make edits. If you deploy the changes and need to make more edits later, simply initiate a new workshop.



##### 1. Add Workshop to a job (specific)

In [4]:
job = lume.Job.get_job_by_id("...")
created_workshop = job.create_workshop()

##### 2. Add Workshop to a pipeline

In [ ]:
pipeline = lume.Pipeline.get_pipeline_by_id("...")
pipeline_workshop = (
    pipeline.create_workshop()
)  # must have already completed jobs associated

##### 3. Retreive a Workshop by ID or get all Workshop data. 

In [ ]:
list_of_workshops = await lume.WorkShop.get_workshops()  # get all workshops
workshop = await lume.WorkShop.get_by_id("...")  # get workshop by id

### Edit Mapping Logic

New mapped data will be present to pipe downstream
Future uses of the corresponding Pipeline will have the updated Mapper and target schema from the edits.
​

**Different Edits**

Each field under the top-level sections contains:

- **`@run_sample`**: Runs a sample for the workshop with the specified ID. Useful for large changes to many target_fields.
  
- **`@run_mapper`**: Runs the mapper of a workshop with the specified ID. 	
  
- **`@run_target_schema`**: Runs the target schema for the workshop with the specified ID.

- **`@run_prompt`**: Runs the prompt for the workshop with the specified ID, and takes the `target_fields_to_prompt` attribute.	



In [ ]:
workshop_target_path = os.path.join(os.getcwd(), "./data/workshop_target.json")
with open(workshop_target_path) as f:
    workshop_target = json.load(f)

In [ ]:
await workshop.run_sample(sample={...})  # run the workshop with sample data

In [ ]:
await workshop.run_prompt(
    target_fields_to_prompt={"currency": "EURO"}
)  # run the workshop with prompt data

In [ ]:
await workshop.run_target_schema(
    target_schema=workshop_target
)  # run the workshop with target data

In [ ]:
mapper = await lume.Mapper.create(
    name="mapper_workshop", data=source_data, target=workshop_target, description="..."
)
mapper = mapper.mapped_data
await workshop.run_mapper(mapper=mapper)  # run the workshop with mapper data

##### Deploying Your Changes:

After making edits, save them through Deploy Workshop. This will update the Mapper in the corresponding pipeline and job, for use in future jobs.



In [ ]:
workshop.deploy()  # deploy the workshop with new edits